### Advent of Code 2023
Joe Housley

## Day 1 - Trebuchet

In [1]:
with open('input1.txt', 'r') as f:
    input = f.readlines()

In [2]:
number_dict = {'one':'1', 'two':'2', 'three':'3', 'four':'4', 'five':'5', 'six':'6', 'seven':'7', 'eight':'8', 'nine':'9'}

In [3]:
def get_first_number(x):
    # check for a string number starting at the beginning of the string. Return the string with the number replaced. If there is no number then return null
    for i in range(1,len(x)+1):
        if x[i-1].isnumeric():
            return x[i-1]
        try:
            # look at the first i characters of the string
            old_line = x[:i]
            # if it contains a number
            for key in number_dict.keys():
                # then replace it
                new_line = old_line.replace(key,number_dict[key], 1)
                if new_line != old_line:
                    return number_dict[key]
            #otherwise loop and search again with one additional character
        except:
            print("Error Error")
    return -1

In [4]:
def get_last_number(x):
    # check for a string number starting at the end of the string. Return the string with the number replaced. If there is no number then return null
    for i in reversed(range(len(x))):
        if x[i].isnumeric():
            return x[i]
        try:
            # look at the last i characters of the string
            old_line = x[i:]
            # if it contains a number
            for key in number_dict.keys():
                # then replace it
                new_line = old_line.replace(key,number_dict[key])
                if new_line != old_line:
                    return number_dict[key]
            #otherwise loop and search again with one additional character
        except:
            print("Error Error")
    return -1

In [5]:
total_sum = 0
for line in input:
    #forwards
    first = get_first_number(line)
    #backwards
    second = get_last_number(line)
    number = first + second
    total_sum += int(number)
print(total_sum)
            

54249


## Day 2 - Cubes

In [1]:
import pandas as pd
import re

In [2]:
with open('input2.txt', 'r') as f:
    lines = f.readlines()

# Game 1: 1 green, 1 blue, 1 red; 3 green, 1 blue, 1 red; 4 green, 3 blue, 1 red; 4 green, 2 blue, 1 red; 3 blue, 3 green
game_ids = []
all_results = []
for line in lines:
    round_results = []
    # extract the game ID
    game_id = int(re.search(r'^Game ([0-9]+): ', line).group(1))
    game_ids.append(game_id)
    # extract the results
    rounds = line.split(';')
    rounds[0] = rounds[0].split(':')[1]
    for x in rounds:
        color_number_dict = {'green':0 , 'blue':0, 'red':0}
        results = x.split(',')
        for result in results:
            match = re.search('([0-9]+) ([a-z]+)', result)
            color = match.group(2)
            number = int(match.group(1))
            color_number_dict[color] = number
        round_results.append(color_number_dict)
    all_results.append(round_results)

In [3]:
data = {'game_id':game_ids, 'results':all_results}
df = pd.DataFrame(data)
df = df.explode('results')
df2 = pd.json_normalize(df['results'])
df = df.reset_index()
df = df.join(df2)
df = df[['game_id','green','blue','red']]

In [4]:
# The Elf would first like to know which games would have been possible if the bag contained only 12 red cubes, 13 green cubes, and 14 blue cubes?
# red 12
# green 13
# blue 14

# filter the df to match the criteria
filtered_df = df[(df['red'] > 12) | (df['blue'] > 14) | (df['green'] > 13)]
bad_games = set(filtered_df['game_id'].unique())
possible_games = set(range(1,101)) - bad_games
sum(possible_games)

2377

In [5]:
powers = []
for i in range(1,101):
    game_df = df[df['game_id'] == i]
    min_green = game_df['green'].max()
    min_blue = game_df['blue'].max()
    min_red = game_df['red'].max()
    powers.append(min_green * min_blue * min_red)
    
sum(powers)

71220

## Day 3 - Gondola

In [80]:
with open('input3.txt','r') as file:
    schematic = file.readlines()

In [76]:
schematic =  ["..........",
              ".........1",
              "..........\n"]

In [73]:
def is_symbol(x):
    if x.isnumeric():
        return False
    if x == '.':
        return False
    return True

In [85]:
total_sum = 0
part_numbers = set()
gears = dict()
for i in range(len(schematic)):
    line = schematic[i].strip()
    j = 0
    while j < len(line):
        # search for a number
        if line[j].isnumeric():
            number = line[j]
            # find the length of the number
            for k in range(j+1,len(line)):
                if line[k].isnumeric():
                    number += line[k]
                else:
                    break
            # HERE WE HAVE FOUND THE WHOLE NUMBER
            # NOW SEARCH FOR AN ADJACENT SYMBOL
            is_adjacent = False
            try:
                if j > 0 and is_symbol(line[j-1]): # look left
                    is_adjacent = True
                    if line[j-1] == "*": # if its a gear
                        location = (i,j-1)
                        if location in gears.keys():
                            gears[location].append(number)
                        else:
                            gears[location] = [number]
                if j+len(number) < len(line) and is_symbol(line[j+len(number)]): # look right
                    is_adjacent = True
                    if line[j+len(number)] == "*": # if its a gear
                        location = (i,j+len(number))
                        if location in gears.keys():
                            gears[location].append(number)
                        else:
                            gears[location] = [number]
                if i != 0: # look above
                    temp_line = schematic[i-1].strip()
                    for k in range(j-1,j+len(number)+1):
                        if k >= 0 and k < len(temp_line) and is_symbol(temp_line[k]):
                            is_adjacent=True
                            if temp_line[k] == "*": # if its a gear
                                location = (i-1,k)
                                if location in gears.keys():
                                    gears[location].append(number)
                                else:
                                    gears[location] = [number]
                if i != len(schematic)-1: # look below
                    temp_line = schematic[i+1].strip()
                    for k in range(j-1,j+len(number)+1):
                        if k >= 0 and k < len(temp_line) and is_symbol(temp_line[k]):
                            is_adjacent=True
                            if temp_line[k] == "*": # if its a gear
                                location = (i+1,k)
                                if location in gears.keys():
                                    gears[location].append(number)
                                else:
                                    gears[location] = [number]
            except IndexError as e:
                print("everything is working as intended still")
                
            if is_adjacent:
                total_sum += int(number)
                part_numbers.add(int(number))
                
            # jump j forward
            j += len(number)
        else:
            j += 1
print(total_sum, sum(part_numbers))

529618 334781


In [89]:
gear_sum = 0
for key,value in gears.items():
    if len(value) == 2:
        gear_sum += int(value[0])*int(value[1])
gear_sum

77509019

## Day 4 - Scratchcards

In [12]:
with open("input4.txt", 'r') as file:
    scratchcards = file.readlines()

In [13]:
def find_winners(card):
    winning_numbers, my_numbers = card.split("|")
    winning_numbers = winning_numbers.split(":")[1].strip()
    winning_numbers = winning_numbers.split()
    #print(winning_numbers)
    
    my_numbers = my_numbers.strip()
    my_numbers = my_numbers.split()
    #print(my_numbers)
    
    points_won = 0
    for number in my_numbers:
        if number in winning_numbers:
            points_won += 1
    return points_won

In [14]:
cards_won = dict.fromkeys([num for num in range(len(scratchcards))], 1)
    
for i,card in enumerate(scratchcards):
    
    points_won = find_winners(card)
    for j in range(points_won):
        cards_won[i+j+1] += cards_won[i]
    
print(sum(cards_won.values()))

9236992


## Day 5 - Gardening

In [10]:
import numpy as np
import pandas as pd

In [1]:
with open('input5-1.txt', 'r') as file:
    almanac = file.readlines()

In [2]:
with open('input5.txt', 'r') as file:
    almanac = file.readlines()

In [4]:
# parse the input
curr_line = 0
# first line contains the seeds
seed_range = [int(seed) for seed in almanac[curr_line].split(':')[1].split()]
seeds = []
for i in [2*x for x in range(int(len(seed_range)/2))]:
    seed_start = seed_range[i]
    seed_len = seed_range[i+1]
    seeds.append((seed_start, seed_start+seed_len))
curr_line += 2
# parse the seed_to_soil map
if almanac[curr_line].split(':')[0] != "seed-to-soil map":
    print("Expected seed-to-soil map. Got",almanac[curr_line].split(':')[0])
curr_line += 1 # move the cursor to the data
seed_to_soil = {} # dict of tuples {seed: (soil, length)}
while curr_line < len(almanac):
    if almanac[curr_line] == '\n':
        curr_line += 1
        break
    # destination    source    length
    dest, src, length = [int(item) for item in almanac[curr_line].split()]
    seed_to_soil[src] = (dest, length)
    curr_line += 1
    
# parse the soil_to_fertilizer map
if almanac[curr_line].split(':')[0] != "soil-to-fertilizer map":
    print("Expected soil-to-fertilizer map. Got",almanac[curr_line].split(':')[0])
curr_line += 1 # move the cursor to the data
soil_to_fertilizer = {} # dict of tuples {seed: (soil, length)}
while curr_line < len(almanac):
    if almanac[curr_line] == '\n':
        curr_line += 1
        break
    # destination    source    length
    dest, src, length = [int(item) for item in almanac[curr_line].split()]
    soil_to_fertilizer[src] = (dest, length)
    curr_line += 1
# parse the fertilizer_to_water map
if almanac[curr_line].split(':')[0] != "fertilizer-to-water map":
    print("Expected fertlizer-to-water map. Got",almanac[curr_line].split(':')[0])
curr_line += 1 # move the cursor to the data
fertilizer_to_water = {} # dict of tuples {seed: (soil, length)}
while curr_line < len(almanac):
    if almanac[curr_line] == '\n':
        curr_line += 1
        break
    # destination    source    length
    dest, src, length = [int(item) for item in almanac[curr_line].split()]
    fertilizer_to_water[src] = (dest, length)
    curr_line += 1
# parse the water_to_light map
if almanac[curr_line].split(':')[0] != "water-to-light map":
    print("Expected water-to-light map. Got",almanac[curr_line].split(':')[0])
curr_line += 1 # move the cursor to the data
water_to_light = {} # dict of tuples {seed: (soil, length)}
while curr_line < len(almanac):
    if almanac[curr_line] == '\n':
        curr_line += 1
        break
    # destination    source    length
    dest, src, length = [int(item) for item in almanac[curr_line].split()]
    water_to_light[src] = (dest, length)
    curr_line += 1
# parse the light_to_temperature map
if almanac[curr_line].split(':')[0] != "light-to-temperature map":
    print("Expected light-to-temperature map. Got",almanac[curr_line].split(':')[0])
curr_line += 1 # move the cursor to the data
light_to_temperature = {} # dict of tuples {seed: (soil, length)}
while curr_line < len(almanac):
    if almanac[curr_line] == '\n':
        curr_line += 1
        break
    # destination    source    length
    dest, src, length = [int(item) for item in almanac[curr_line].split()]
    light_to_temperature[src] = (dest, length)
    curr_line += 1
# parse the temperature_to_humidity map
if almanac[curr_line].split(':')[0] != "temperature-to-humidity map":
    print("Expected temperature-to-humidity map. Got",almanac[curr_line].split(':')[0])
curr_line += 1 # move the cursor to the data
temperature_to_humidity = {} # dict of tuples {seed: (soil, length)}
while curr_line < len(almanac):
    if almanac[curr_line] == '\n':
        curr_line += 1
        break
    # destination    source    length
    dest, src, length = [int(item) for item in almanac[curr_line].split()]
    temperature_to_humidity[src] = (dest, length)
    curr_line += 1
# parse the humidity_to_location map
if almanac[curr_line].split(':')[0] != "humidity-to-location map":
    print("Expected humidity-to-location map. Got",almanac[curr_line].split(':')[0])
curr_line += 1 # move the cursor to the data
humidity_to_location = {} # dict of tuples {seed: (soil, length)}
while curr_line < len(almanac):
    if almanac[curr_line] == '\n':
        curr_line += 1
        break
    # destination    source    length
    dest, src, length = [int(item) for item in almanac[curr_line].split()]
    humidity_to_location[src] = (dest, length)
    curr_line += 1


In [3]:
def lookup_value(src, src_dest_map):
    for src_start in src_dest_map.keys():
        dest_start, length = src_dest_map[src_start]
        if src >= src_start and src < src_start + length:
            # figure out how far from the start the seed is
            diff = src-src_start
            # then go that distance from soil_start
            return dest_start + diff
    return src

In [19]:
def interval_intersect(a,b):
    a0,a1 = a
    b0,b1 = b
    if a1 < b0 or b1 < a0: # does not intersect
        return None
    else:
        return (max(a0,b0), min(a1,b1))
print(interval_intersect((0,9),(2,7)))
def lookup_value(seed_sections, seed_soil_map):
    soil_sections = []
    for seed_range in seed_sections:
        for seed_start in seed_soil_map.keys():
            soil_start, length = seed_soil_map[seed_start]
            seed_end = seed_start + length
            map_range = (seed_start, seed_end)
            intersection = interval_intersect(seed_range, map_range)
            if intersection is None:
                soil_sections.append(seed_range) # there are no seeds in the almanac
            else:
                seed_lower = min(seed_start, seed_range[0])
                seed_upper = max(seed_end, seed_range[1])
                if seed_lower < intersection[0]:
                    soil_sections.append((seed_lower,intersection[0])) # add the part of the interval that is not in the almanac
                if seed_upper > intersection[0]:
                    soil_sections.append((intersection[1], seed_upper)) # add the part of the interval that is not in the almanac
                # now add the intersection
                # find the offset
                offset = intersection[0] - seed_start
                # now move the soil by the offset
                soil_range = (soil_start + offset, soil_start + length + offset)
                soil_sections.append(soil_range)
    return list(set(soil_sections))

lookup_range([(0,2),(1,3)], {1:(3,4)})
        

(2, 7)


[(0, 1), (3, 7), (2, 5), (3, 5)]

In [17]:
seeds

[(1482445116, 1821632509),
 (3210489476, 3722395312),
 (42566461, 94415598),
 (256584102, 636159946),
 (3040181568, 3180147594),
 (4018529087, 4135337336),
 (2887351536, 2976867314),
 (669731009, 1476619499),
 (2369242654, 2859166585),
 (2086168596, 2169059849)]

In [24]:
locations = []
for seed in seeds:
    # seed -> soil
    soil = lookup_value([seed], seed_to_soil)
    # soil -> fertilizer
    fertilizer = lookup_value(soil, soil_to_fertilizer)
    # fertilizer -> water
    water = lookup_value(fertilizer, fertilizer_to_water)
    # water -> light
    light = lookup_value(water, water_to_light)
    # light -> temperature
    temperature = lookup_value(light, light_to_temperature)
    # temperature -> humidity
    humidity = lookup_value(temperature, temperature_to_humidity)
    # humidity => location    
    location = lookup_value(humidity, humidity_to_location)
    locations.append(location)
    #print(seed,soil,fertilizer,water,light,temperature,humidity,location)
lower_loc = [loc[0] for loc in locations]
print(min(lower_loc))

(970847216, 1246226539)


In [3]:
import sys
import re
from collections import defaultdict
D = open("input5.txt").read().strip()
L = D.split('\n')

parts = D.split('\n\n')
seed, *others = parts
seed = [int(x) for x in seed.split(':')[1].split()]

class Function:
    def __init__(self, S):
        lines = S.split('\n')[1:] # throw away name
        # dst src sz
        self.tuples: list[tuple[int,int,int]] = [[int(x) for x in line.split()] for line in lines]
        #print(self.tuples)
    def apply_one(self, x: int) -> int:
        for (dst, src, sz) in self.tuples:
            if src<=x<src+sz:
                return x+dst-src
        return x
  
    # list of [start, end) ranges
    def apply_range(self, R):
        A = []
        for (dest, src, sz) in self.tuples:
            src_end = src+sz
            NR = []
            while R:
                # [st                                     ed)
                #          [src       src_end]
                # [BEFORE ][INTER            ][AFTER        )
                (st,ed) = R.pop()
                # (src,sz) might cut (st,ed)
                before = (st,min(ed,src))
                inter = (max(st, src), min(src_end, ed))
                after = (max(src_end, st), ed)
                if before[1]>before[0]:
                    NR.append(before)
                if inter[1]>inter[0]:
                    A.append((inter[0]-src+dest, inter[1]-src+dest))
                if after[1]>after[0]:
                    NR.append(after)
            R = NR
        return A+R

Fs = [Function(s) for s in others]

def f(R, o):
    A = []
    for line in o:
        dest,src,sz = [int(x) for x in line.split()]
        src_end = src+sz

P1 = []
for x in seed:
    for f in Fs:
        x = f.apply_one(x)
    P1.append(x)
print(min(P1))

P2 = []
pairs = list(zip(seed[::2], seed[1::2]))
for st, sz in pairs:
    # inclusive on the left, exclusive on the right
    # e.g. [1,3) = [1,2]
    # length of [a,b) = b-a
    # [a,b) + [b,c) = [a,c)
    R = [(st, st+sz)]
    for f in Fs:
        R = f.apply_range(R)
    #print(len(R))
    P2.append(min(R)[0])
print(min(P2))

175622908
5200543


## Day 6 - Boat Races

In [2]:
with open('input6.txt', 'r') as file:
    race_information = file.readlines()

times = race_information[0].split(':')[1].strip().split()
distances = race_information[1].split(':')[1].strip().split()

print([(t,d) for t,d in zip(times,distances)])

[('58', '434'), ('81', '1041'), ('96', '2219'), ('76', '1218')]


In [3]:
def calc_distance_traveled(speed, time_remaining):
    return speed * time_remaining

records_broken = 1
for race_num in range(len(times)):
    time = int(times[race_num])
    record = int(distances[race_num])
    record_breaking = 0
    for speed in range(time): # for each discrete time interval
        # speed is time passed
        time_left = time - speed
        distance_traveled = calc_distance_traveled(speed, time_left)
        if distance_traveled > record:
            record_breaking += 1
    records_broken *= record_breaking
print(records_broken)

1159152


In [8]:
time = int(race_information[0].split(':')[1].strip().replace(' ',''))
record = int(race_information[1].split(':')[1].strip().replace(' ',''))

record_breaking = 0
for speed in range(time): # for each discrete time interval
    # speed is time passed
    time_left = time - speed
    distance_traveled = calc_distance_traveled(speed, time_left)
    if distance_traveled > record:
        record_breaking += 1
print(record_breaking)

41513103


## Day 7 - Camel Cards

In [1]:
import pandas as pd
hands = pd.read_csv('input7.txt', ' ', header=None)
card_value = {'A':14,'K':13,'Q':12,'J':0,'T':10,'9':9,'8':8,'7':7,'6':6,'5':5,'4':4,'3':3,'2':2}
hands.columns = ['cards','bid']
hands['first'] = hands['cards'].apply(lambda x: x[0]).map(card_value)
hands['second'] = hands['cards'].apply(lambda x: x[1]).map(card_value)
hands['third'] = hands['cards'].apply(lambda x: x[2]).map(card_value)
hands['fourth'] = hands['cards'].apply(lambda x: x[3]).map(card_value)
hands['fifth'] = hands['cards'].apply(lambda x: x[4]).map(card_value)
hands

,cards,bid,first,second,third,fourth,fifth
0,AJ44J,454,14,0,4,4,0
1,33848,56,3,3,8,4,8
2,66366,699,6,6,3,6,6
3,KQKJK,718,13,12,13,0,13
4,47767,78,4,7,7,6,7
...,...,...,...,...,...,...,...
995,2Q777,972,2,12,7,7,7
996,J876T,800,0,8,7,6,10
997,Q5JQQ,656,12,5,0,12,12
998,JJAJT,817,0,0,14,0,10


In [9]:
# strongest to weakest
# 7 Five of a Kind: AAAAA
# 6 Four of a Kind: AAKAA
# 5 Full House: AAAKK
# 4 Three of a Kind: AAAKQ
# 3 Two Pair: AAKKQ
# 2 One Pair: AAKQJ
# 1 High Card AKQJ9
def rank_key(hand):
    # five of a kind
    first,second,third,fourth,fifth = hand
    card_count = dict.fromkeys(['A','K','Q','T','9','8','7','6','5','4','3','2'],0)
    for card in hand:
        if card == 'J':
            return 0
        card_count[card] += 1
    if 5 in card_count.values(): # five of a kind
        return 7
    if 4 in card_count.values(): # four of a kind
        return 6
    if 3 in card_count.values():
        if 2 in card_count.values():
            return 5
        else:
            return 4
    if 2 in card_count.values():
        vals = list(card_count.values())
        vals.remove(2)
        if 2 in vals:
            return 3
        else:
            return 2
    return 1

def joker_rank(hand):
    # Assuming J is a wildcard, find the best score for this hand
    cards = ['A','K','Q','T','9','8','7','6','5','4','3','2']
    if 'J' in hand:
        return max([joker_rank(hand.replace('J', c, 1)) for c in cards])
    return rank_key(hand)

joker_rank('AAAAJ')

7

In [10]:
hands['value'] = hands['cards'].apply(joker_rank)
hands.sort_values(['value','first','second','third','fourth','fifth'], inplace=True)
hands = hands.reset_index()
hands.index += 1
hands['winnings'] = hands.index * hands['bid']
hands

,level_0,index,cards,bid,first,second,third,fourth,fifth,value,winnings
1,339,211,246Q3,276,2,4,6,12,3,1,276
2,340,388,25673,334,2,5,6,7,3,1,668
3,341,691,257T9,533,2,5,7,10,9,1,1599
4,342,840,25QT4,564,2,5,12,10,4,1,2256
5,343,620,2835A,869,2,8,3,5,14,1,4345
...,...,...,...,...,...,...,...,...,...,...,...
996,265,115,TTJTT,190,10,10,0,10,10,7,189240
997,266,187,TTTJJ,569,10,10,10,0,0,7,567293
998,295,12,QQQQJ,387,12,12,12,12,0,7,386226
999,314,450,KKKJJ,814,13,13,13,0,0,7,813186


In [11]:
hands['winnings'].sum()

252127335

## Day 8